In [5]:
import pandas as pd
import pickle
import json
from tqdm import tqdm
import time
import pickle
from yuno1.ui import base as uBase
from yuno1.search import base as sBase
import PIL
from yuno.preprocessing.filter import *
from crawler import *

In [6]:
anime_info_dict= {}

In [7]:
data = pd.read_csv('data/anime_data.csv')

In [8]:
def get_data(index):
    json_data = json.loads(data['json'][index])
    uid = json_data['id']
    names = list(json_data["title"].values())
    img_url = json_data["coverImage"]["large"]
    
    synopsis = json_data["description"]
    anilist_url = json_data["siteUrl"]
    mal_url = "https://myanimelist.net/anime/"+str(uid)
    anime_info_dict[sBase.AnimeUid(uid)] = uBase.AnimeInfo(names,img_url,synopsis,anilist_url,mal_url)

In [9]:
for index in range(len(data['id'])):
    get_data(index)

In [23]:
print(f"Number of anime info records: {len(anime_info_dict)}")

Number of anime info records: 17773


In [11]:
anime_infos = {int:AnimeInfo}

In [12]:
#List containing character failed to process  for later review
false_character = []

In [15]:
def get_ani_data(index):
    json_data = json.loads(data['json'][index])
    
    uid = json_data['idMal']
    
    if not uid:
        return
    
    titles = get_anime_title(json_data)
    
    characters = []
    character_json = json_data["characters"]["edges"]
    #For each anime iterate through all characters
    for char_id in range(len(character_json)):
        
        #Get character names
        character_names = Char_Helper.get_character_names(character_json,char_id)
        if len(character_names)<1:
            continue
        
        #Fetch character gender
        gender = Char_Helper.get_character_gender(character_json,char_id)
        if gender is None:
            continue
        
        #If both characters name and gender(MALE,FEMALE) is valid, create character info then add it to anime info
        char = Character(character_names, gender)
        characters.append(char)
    
    ani_info = AnimeInfo(uid,titles,characters)
    anime_infos[sBase.AnimeUid(uid)] = ani_info

In [16]:
for index in tqdm(range(len(data['id']))):
    get_ani_data(index)

100%|██████████| 17773/17773 [00:03<00:00, 5365.66it/s]


In [ ]:
with open('data/anime_info1.pickle', 'wb') as f:
    pickle.dump(anime_infos, f)

In [17]:
print(len(false_character))

0


In [20]:
print(anime_infos[34096].characters)

[Character(names=['Zura', 'Fruit Punch Samurai', 'Zurako', 'Captain Katsura', 'Katsuo', 'ObaZ', 'Kotarou'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Yorozuya', 'Mr. Odd Jobs', 'Shiroyasha', 'Gin', 'Gin-chan', 'Kintoki', 'Paako', 'Ginpachi-sensei', 'Ginnoji', 'GFundz', 'Gintoki'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Gura', 'China Girl', 'Leader', 'Gurako', 'Kagura'], gender=<Gender.FEMALE: '[FEMALE_CHAR]'>), Character(names=['Shin-chan', 'Shin', 'Pattsuan', 'Megane', 'Pachie', 'Shintank', 'Watson', 'Pasta', 'Shinpachi'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Admiral idiot', 'Baka Teitoku', 'Baka Aniki', 'Kamui'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Nobutasu', 'Nobume'], gender=<Gender.FEMALE: '[FEMALE_CHAR]'>), Character(names=['Mucchi', 'Mutsu'], gender=<Gender.FEMALE: '[FEMALE_CHAR]'>), Character(names=['Utsuro'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Master of Kouan', 'Kouka'], gender=<Gender.F

In [21]:
anime_infos[34096]

AnimeInfo(uid=34096, names=['銀魂。', 'Gintama', 'Gintama Season 5'], characters=[Character(names=['Zura', 'Fruit Punch Samurai', 'Zurako', 'Captain Katsura', 'Katsuo', 'ObaZ', 'Kotarou'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Yorozuya', 'Mr. Odd Jobs', 'Shiroyasha', 'Gin', 'Gin-chan', 'Kintoki', 'Paako', 'Ginpachi-sensei', 'Ginnoji', 'GFundz', 'Gintoki'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Gura', 'China Girl', 'Leader', 'Gurako', 'Kagura'], gender=<Gender.FEMALE: '[FEMALE_CHAR]'>), Character(names=['Shin-chan', 'Shin', 'Pattsuan', 'Megane', 'Pachie', 'Shintank', 'Watson', 'Pasta', 'Shinpachi'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Admiral idiot', 'Baka Teitoku', 'Baka Aniki', 'Kamui'], gender=<Gender.MALE: '[MALE_CHAR]'>), Character(names=['Nobutasu', 'Nobume'], gender=<Gender.FEMALE: '[FEMALE_CHAR]'>), Character(names=['Mucchi', 'Mutsu'], gender=<Gender.FEMALE: '[FEMALE_CHAR]'>), Character(names=['Utsuro'], gender=<Gender.MALE: '